# Responsável por tratar os dados pertinente a cliente

## Importando bibliotecas

In [10]:
import sys, os

# Carregando pacote que estao fora de /transform
path_raiz = os.path.abspath(os.path.join("..", ".."))
sys.path.append(path_raiz)

In [11]:
import pandas as pd
from src.utils import carregar_tabela_excel
from src.insert_client.insert_client import inserir_cliente
import json

## Importando e tratando os dados

In [12]:
source_input = "../../data/baseTesteIXC_Transform.xlsx"
source_output = "../../src/inserted/cliente_insert.xlsx"

In [13]:
# Carregar o DataFrame com as colunas especificadas como strings
df = pd.read_excel(source_input, dtype={
    'cpf_cnpj': str,
    'cep': str,
    'numero': str,
    # Adicione outras colunas conforme necessário
})

In [14]:
# Lista das colunas que fazem sentido enviar para a API
colunas_para_api = [
    'razaoSocial',
    'Email',
    'Celular',
    'cpf_cnpj',
    'rg_ie',
    'filial_id',
    'idCidade',
    'Endereco',
    'Complemento',
    'Numero',
    'Bairro',
    'Cep',
]

# Filtra o DataFrame para manter apenas essas colunas
df = df[colunas_para_api]


### Inserção teste

In [15]:
# inserir_cliente(
#     razao="Cliente de Teste 2",
#     cnpj_cpf="28086842000127",
#     cep="57160-000",
#     endereco="Rua ABC",
#     numero="123",
#     bairro="Bairro XPTO",
#     filial_id="35",
#     cidade="1659",
#     uf="AL",
#     telefone_celular="82999999999",
#     email="teste@example.com",
# )

## Insersão do DF

In [16]:
# Limitando o DF
df = df.head(10)
df

,razaoSocial,Email,Celular,cpf_cnpj,rg_ie,filial_id,idCidade,Endereco,Complemento,Numero,Bairro,Cep
0,Abelanio Veríssimo Peixoto,teste@gmail.com,99999999999,685.796.740-77,Isento,35,1659,Vila São Sebastião,Próximo ao cacimbão,0,Centro,57830-000
1,Aclecia da Silva,teste@gmail.com,99999999999,281.742.620-75,30234875,35,1659,Maria de Lurdes,Próximo ao mirante,SN,Centro,57830-000
2,Adeilton da Mata,teste@gmail.com,99999999999,673.248.950-00,40259595,35,1659,Ass. Nova Esperança,próximo a casa de farinha,SN,Zona Rural,57830-000
3,Adeilton da Silva,teste@gmail.com,99999999999,108.151.210-59,3614522-0,35,1659,Gravatá,Lote do Sr. Del,SN,Zona Rural,57830-000
4,Adeilton Teixeira de Lima,teste@gmail.com,99999999999,976.069.770-03,36145220,35,1659,João Lyra / Qd: A,NaN,24,Conj. Raimundo Nonato Lopes,57830-000
5,Adelmo Luiz Berto De Oliveira,teste@gmail.com,99999999999,660.107.170-60,ISENTO,35,1659,Rua: Manoel Luiz da Silva,Próximo ao Mirante,SN,Centro,57830-000
6,Ademar Manoel da Silva,teste@gmail.com,99999999999,52.459.301/0001-81,30972841,35,1659,Ass.: Flor do Mundaú,NaN,SN,Zona Rural,57830-000
7,Adriana Paulino da Silva,teste@gmail.com,99999999999,54.404.537/0001-46,ISENTO,35,1659,Platô 1/ Q: 02,NaN,06,Conj. Raimundo Nonato Lopes,57830-000


In [17]:
# Crie as colunas adicionais (inicialmente vazias)
df["idClienteIXC"] = None
df["statusInsercao"] = None
df["logRetorno"] = None

for idx, row in df.iterrows():
    # Mapeamos as colunas do DF para os parâmetros da função inserir_cliente.
    razao = row["razaoSocial"]
    email = row["Email"]
    celular = row["Celular"]
    cnpj_cpf = row["cpf_cnpj"]
    rg_ie = row["rg_ie"]
    filial = row["filial_id"]
    cidade = row["idCidade"] 
    endereco = row["Endereco"]
    complemento = row["Complemento"]
    numero = row["Numero"]
    bairro = row["Bairro"]
    cep = row["Cep"]
    
    # Chamamos a função inserir_cliente:
    response = inserir_cliente(
        razao=razao,
        cnpj_cpf=cnpj_cpf,
        cep=cep,
        endereco=endereco,
        numero=numero,
        complemento=complemento,
        bairro=bairro,
        cidade=cidade,
        filial_id=filial,
        telefone_celular=celular,
        email=email,
        ie_identidade=rg_ie,
        # Se você quiser passar outros campos com valores-padrão, inclua aqui
        # Ex.: uf="AL" etc.
    )
    
    status_code = response.status_code
    content = response.content  # bytes

    # Fazemos parse do JSON:
    # b'{"type":"success","message":"Registro inserido com sucesso!","id":"117580", ...}'
    json_resp = json.loads(content.decode("utf-8", errors="replace"))

    if status_code == 200 and json_resp.get("type") == "success":
        # Sucesso: capturamos o "id"
        client_id = json_resp.get("id")  # Exemplo: "117580"
        
        df.at[idx, "idClienteIXC"] = client_id
        df.at[idx, "statusInsercao"] = "sucesso"
        # logRetorno pode ficar vazio ou None em caso de sucesso
        df.at[idx, "logRetorno"] = json_resp
    else:
        # Falha: salvamos o log completo
        df.at[idx, "idClienteIXC"] = None
        df.at[idx, "statusInsercao"] = "falha"
        # Guardamos o JSON completo (ou a chave "message", se preferir):
        df.at[idx, "logRetorno"] = json_resp

200 OK
b'{"type":"error","message":"Preencha Tipo pessoa","atualiza_campos":[{"tipo":"r","campo":"ativo","valor":"S"},{"tipo":"i","campo":"data_cadastro","valor":""},{"tipo":"i","campo":"filial_id","valor":"35"},{"tipo":"i","campo":"latitude","valor":""},{"tipo":"i","campo":"longitude","valor":""},{"tipo":"i","campo":"id_conta","valor":""},{"tipo":"d","campo":"crm_data_vencemos","valor":""},{"tipo":"r","campo":"convert_cliente_forn","valor":""},{"tipo":"d","campo":"crm_data_perdemos","valor":""},{"tipo":"i","campo":"crm_data_sem_viabilidade","valor":""},{"tipo":"i","campo":"crm_data_sem_porta_disponivel","valor":""},{"tipo":"i","campo":"crm_data_abortamos","valor":""},{"tipo":"i","campo":"crm_data_negociando","valor":""},{"tipo":"i","campo":"crm_data_apresentando","valor":""},{"tipo":"i","campo":"crm_data_sondagem","valor":""},{"tipo":"i","campo":"crm_data_novo","valor":""}]}'
200 OK
b'{"type":"error","message":"Preencha Tipo pessoa","atualiza_campos":[{"tipo":"r","campo":"ativo","valo

### Retorno

In [19]:
df.to_excel(source_output)